# Loading Data

In [1]:
from data import load_data
import numpy as np

clinical, _, genefpkm, treatment, outcome = load_data()

print(clinical.shape)

clinical.describe().T

(724, 35)


,count,mean,std,min,25%,50%,75%,max
cmmc,376.0,5907.877660,9307.921203,0.00000,233.25000,1409.50000,7145.500000,47146.00000
ecog_ps,357.0,1.330532,0.638197,1.00000,1.00000,1.00000,1.000000,4.00000
percent_aneuploid,539.0,18.178948,21.639551,0.00000,0.00000,10.00000,29.700000,89.40000
percent_plama_cells_bone_marrow,616.0,17.599838,16.658980,0.00000,5.70000,11.60000,23.225000,84.30000
percent_plama_cells_peripherical_blood,616.0,0.636519,2.831528,0.00000,0.00000,0.00000,0.100000,33.30000
creatinine,710.0,106.869476,62.670669,33.00000,70.72000,88.40000,116.688000,503.88000
iss,704.0,1.917614,0.799379,1.00000,1.00000,2.00000,3.000000,3.00000
absolute_neutrophil,707.0,3.912428,2.173153,0.58000,2.40000,3.50000,4.760000,16.51200
platelet,723.0,220.117566,79.182326,18.00000,167.00000,215.00000,262.000000,668.00000
wbc_x10_10_9_l,616.0,6.271688,2.517359,1.40000,4.50000,5.90000,7.500000,25.80000


# Gene Expressions

In [2]:
genefpkm.iloc[:8, :8]

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036
ID,,,,,,,,
MMRF1021,959,0,1264,668,244,27,3711,1016
MMRF1024,776,0,972,595,78,504,10,1263
MMRF1029,1470,0,2143,1093,209,36,20,3677
MMRF1030,83,0,1235,422,58,42,21,1714
MMRF1031,2,0,1127,432,190,48,117,2527
MMRF1032,24,0,748,214,62,33,65,571
MMRF1033,18,0,827,478,46,211,5,962
MMRF1037,18,0,870,385,66,10,22,1165


# Treatments

In [3]:
treatment.head()

,therapy_first_line_Bor-Cyc-Dex,therapy_first_line_Bor-Dex,therapy_first_line_Bor-Len-Dex,therapy_first_line_Len-Dex,therapy_first_line_Non-therapy
ID,,,,,
MMRF1021,0,0,1,0,0
MMRF1024,0,0,0,0,1
MMRF1029,0,0,1,0,0
MMRF1030,0,0,1,0,0
MMRF1031,0,0,1,0,0


# THERAPY SENSITIVITY MODELLING

## Cross-validation Experiment

In [12]:
from pipeline import NMLA

from sklearn.model_selection import StratifiedKFold
from evaluation import optimize_threshold, classification_metrics
from sklearn.metrics import roc_auc_score, log_loss, confusion_matrix


from constants import N_FOLDS, RANDOM_STATE
from util import join_values

import lightgbm as lgb
import pickle as pkl
import pandas as pd
import numpy as np
import time
import os

# creating analyser object to compute and group 
# classification matrics grouped by training and validation
# dataset and by experiment id
# analyser = Analyser()

# Creating 10-fold CV splits stratified by treatment and outcome
kfold = StratifiedKFold(N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
split = kfold.split(np.zeros(outcome.shape[0]), join_values([treatment, outcome]))

#
#
result = {c: [] for c in ['experiment', 'train_auc', 'valid_auc', 
                          'train_loss', 'valid_loss', 'execution_time', 'threshold']}

for experiment, (train_index, valid_index) in enumerate(split):
    
    initial_time = time.time()
    
    print('{}\n\n'.format(experiment))
    
    #######################################################################################################
    # Split train & valid
    #######################################################################################################
    
    response_train = outcome.iloc[train_index, 0]
    response_valid = outcome.iloc[valid_index, 0]

    clinical_train = clinical.iloc[train_index, :]
    clinical_valid = clinical.iloc[valid_index, :]

    treatment_train = treatment.iloc[train_index, :]
    treatment_valid = treatment.iloc[valid_index, :]
    
    genefpkm_train = genefpkm.iloc[train_index, :]
    genefpkm_valid = genefpkm.iloc[valid_index, :]
    
    #######################################################################################################
    # Train & Test distances
    #######################################################################################################

    if False:
        
        dists = []

        for row_train in clinical.join(genefpkm, how='inner').values:
            for row_valid in clinical.join(genefpkm, how='inner').values:
                dists.append(np.linalg.norm(row_train-row_valid))

        train_test_distance_avg = np.mean(dists)
        train_test_distance_std = np.std(dists)
        train_test_distance_min = np.min(dists)
        train_test_distance_max = np.max(dists)

    #######################################################################################################
    # NMLA fitting
    #######################################################################################################
    
    nmla = NMLA(experiment_number=experiment, 
                number_of_experiments=N_FOLDS, 
                output_path='./output/nmla/', 
                random_state=RANDOM_STATE)

    nmla.fit(clinical_train, genefpkm_train, treatment_train, response_train, 

        lgb_fixed_parameters = {
            'metric': 'binary_logloss',
            'n_estimators': 100,
            'objective': 'binary',
            'is_unbalance': False, 
            'extra_trees': True,
            'max_depth': 4,
            'learning_rate': 0.1,
            'min_split_gain': 0.0001,
            'min_child_weight': 0.0001},

        optimization_n_call=50,
        optimization_n_folds=2,
        optimization_early_stopping_rounds=1,

        clinical_marker_selection_threshold=0.050,
        genefpkm_marker_selection_threshold=0.002,

        dae_decay_rate=1.0,
        dae_learning_rate=1e-1,
        dae_steps=100000,
        dae_early_stopping_rounds=1000,

        lgb_early_stopping_rounds=1,

        predictor_n_folds=3)

    with open('output/nmla/trained_model_{}.pkl'.format(experiment), 'wb') as file:
        pkl.dump(nmla, file)
    
    #######################################################################################################
    # NMLA inference
    #######################################################################################################

    y_hat_train = nmla.predict(clinical_train, genefpkm_train, treatment_train)
    y_hat_valid = nmla.predict(clinical_valid, genefpkm_valid, treatment_valid)

    #################################################################################################
    # Analysing Performance
    #################################################################################################   

    # Computing AUC
    train_auc = roc_auc_score(response_train, y_hat_train)
    valid_auc = roc_auc_score(response_valid, y_hat_valid)

    # Computing logLoss
    train_loss = log_loss(response_train, y_hat_train)
    valid_loss = log_loss(response_valid, y_hat_valid)

    # Compute optimized threshold
    opt_threshold = optimize_threshold(response_train, y_hat_train)

    if opt_threshold is None:
        opt_threshold = np.mean(response_train)

    # compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(response_valid, [int(y >= opt_threshold) for y in y_hat_valid]).ravel()

    classification_results = classification_metrics(tn, fp, fn, tp)

    # add results to data frame (dict for now)
    for k in classification_results:
        if k not in result:
            result[k] = []
        result[k].append(classification_results[k])

    result['experiment'].append(experiment)
    result['train_auc'].append(train_auc)
    result['valid_auc'].append(valid_auc)
    result['train_loss'].append(train_loss)
    result['valid_loss'].append(valid_loss)
    result['execution_time'].append(time.time() - initial_time)
    result['threshold'].append(opt_threshold)

    print('Experiment {} with {} genes and {} clinical markers'.format(
          experiment, len(nmla.selected_genefpkm[0]), len(nmla.selected_clinical[0])))

    print('Train: {}'.format(train_auc))

    print('Valid: {}'.format(valid_auc))

    print("\n========================================================================================\n")
    
    # Exporting inference
    response_train = pd.DataFrame(response_train)
    response_train['y_hat'] = y_hat_train
    response_train.to_csv('output/nmla/inference/train_{}.csv'.format(experiment), index=True, sep=',')
    
    response_valid = pd.DataFrame(response_valid)
    response_valid['y_hat'] = y_hat_valid
    response_valid.to_csv('output/nmla/inference/valid_{}.csv'.format(experiment), index=True, sep=',')

0


6
54
early stopping after 1000 iterations without improvements with 1868 steps: best metric value 130.2523651123047
INFO:tensorflow:Restoring parameters from C:\Users\Venezian\git\multiple-myeloma\output\nmla\dae\data_augmentation_adadelta_000\graph\data_augmentation_adadelta_000
INFO:tensorflow:Restoring parameters from C:\Users\Venezian\git\multiple-myeloma\output\nmla\dae\data_augmentation_adadelta_000\graph\data_augmentation_adadelta_000
TRAIN mean log loss: 0.5039269686337223
TRAIN mean AUC: 0.7181883079798558
VALID mean log loss: 0.5162642554002138
VALID mean AUC: 0.6778512881454059
INFO:tensorflow:Restoring parameters from C:\Users\Venezian\git\multiple-myeloma\output\nmla\dae\data_augmentation_adadelta_000\graph\data_augmentation_adadelta_000
INFO:tensorflow:Restoring parameters from C:\Users\Venezian\git\multiple-myeloma\output\nmla\dae\data_augmentation_adadelta_000\graph\data_augmentation_adadelta_000
Experiment 0 with 16 genes and 6 clinical markers
Train: 0.71245572609

In [14]:
result = pd.DataFrame(result)

result.to_csv('output/nmla/metrics.csv', sep=',', index=False)

result

,experiment,train_auc,valid_auc,train_loss,valid_loss,execution_time,threshold,accuracy,precision,sensitivity,specificity
0,0,0.712456,0.704361,0.772127,0.639063,136.549313,0.482681,0.706667,0.352941,0.352941,0.810345
1,1,0.743063,0.746999,0.502551,0.503089,226.636832,0.200000,0.618421,0.380952,0.842105,0.543860
2,2,0.797014,0.632502,0.441184,0.626764,200.091617,0.294788,0.578947,0.303030,0.526316,0.596491
3,3,0.743375,0.727183,0.520552,0.550897,152.536349,0.452093,0.729730,0.458333,0.611111,0.767857
4,4,0.809434,0.594949,0.470249,0.609937,126.556769,0.330000,0.561644,0.294118,0.555556,0.563636
5,5,0.690633,0.697326,1.977342,2.473817,253.840475,0.350000,0.694444,0.391304,0.529412,0.745455
6,6,0.778399,0.733690,0.525325,0.536358,125.230131,0.441281,0.694444,0.400000,0.588235,0.727273
7,7,0.712611,0.675926,0.730144,1.239241,237.737613,0.310000,0.565217,0.325581,0.933333,0.462963
8,8,0.772474,0.674292,0.508688,0.603379,128.446931,0.391122,0.690141,0.400000,0.588235,0.722222
9,9,0.753366,0.679945,0.577517,0.640395,402.866185,0.380817,0.621212,0.310345,0.642857,0.615385


In [15]:
result.mean()

experiment          4.500000
train_auc           0.751282
valid_auc           0.686717
train_loss          0.702568
valid_loss          0.842294
execution_time    199.049221
threshold           0.363278
accuracy            0.646087
precision           0.361661
sensitivity         0.617010
specificity         0.655549
dtype: float64

In [17]:
result.std()

experiment         3.027650
train_auc          0.038691
valid_auc          0.046642
train_loss         0.460763
valid_loss         0.610336
execution_time    87.280629
threshold          0.084843
accuracy           0.063984
precision          0.053278
sensitivity        0.164146
specificity        0.114193
dtype: float64

## Analyse per Treatment

In [63]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

def classification_metrics(x):
    try:
        
        tn, fp, fn, tp = confusion_matrix(x['y_true'], x['y_hat_hard']).ravel()
        
        sensitivity, specificity = tp / (tp + fn), tn / (tn + fp)
        
        precision = tp / (tp + fp)
        
        return pd.Series({
            'AUC': roc_auc_score(x['y_true'], x['y_hat_adjusted']),
            'Accuracy': accuracy_score(x['y_true'], x['y_hat_hard']),
            'Sensitivity': sensitivity,
            'Specificity': specificity,
            'Precision': precision,
            'F1': 2 * precision * sensitivity / (precision + sensitivity)})
    
    except:
        return pd.Series({
            'AUC': 0.0,
            'Accuracy': 0.0,
            'Sensitivity': 0.0,
            'Specificity': 0.0,
            'Precision': 0.0,
            'F1': 0.0})
    
BASE_DIR = 'output/nmla/inference/'
    
df = None

metrics = pd.read_csv('output/nmla/metrics.csv', sep=',')

for file in os.listdir(BASE_DIR):
    if 'valid' in file:
        index = file.split('_')[1].split('.')[0]
        train = pd.read_csv(os.path.join(BASE_DIR, 'train_{}.csv'.format(index)), sep=',')
        t = metrics[metrics['experiment'] == int(index)]['threshold'].values[0]
        tmp = pd.read_csv(os.path.join(BASE_DIR, file), sep=',')
        df = tmp if df is None else pd.concat([df, tmp], axis=0)
        df['y_hat_hard'] = df['y_hat'].apply(lambda x: 1 if x >= t else 0)
        
df = df.set_index('ID', drop=True)

df = df.join(treatment[treatment==1].stack().reset_index().drop(0,1).set_index('ID').rename(columns={'level_1': 'treatment'}))

df = df.rename(columns={'response_best_response_first_line': 'y_true'})

df.groupby('treatment').apply(classification_metrics)

,AUC,Accuracy,Sensitivity,Specificity,Precision,F1
treatment,,,,,,
therapy_first_line_Bor-Cyc-Dex,0.630968,0.631579,0.551724,0.653846,0.307692,0.395062
therapy_first_line_Bor-Dex,0.654135,0.765625,0.571429,0.789474,0.250000,0.347826
therapy_first_line_Bor-Len-Dex,0.671300,0.601695,0.646154,0.584795,0.371681,0.471910
therapy_first_line_Len-Dex,0.666113,0.780000,0.285714,0.860465,0.250000,0.266667
therapy_first_line_Non-therapy,0.659388,0.637931,0.555556,0.668639,0.384615,0.454545


In [65]:
df.groupby('treatment')['y_true'].count()

treatment
therapy_first_line_Bor-Cyc-Dex    133
therapy_first_line_Bor-Dex         64
therapy_first_line_Bor-Len-Dex    236
therapy_first_line_Len-Dex         50
therapy_first_line_Non-therapy    232
Name: y_true, dtype: int64

## Statistical Test SMLA and MLA

,Unnamed: 0,experiment,predictor,train_auc,valid_auc,train_loss,valid_loss,execution_time,threshold,accuracy,precision,sensitivity,specificity
0,0,0,mlp,0.775180,0.622718,0.467782,0.534754,151.593535,0.201045,0.600000,0.290323,0.529412,0.620690
1,1,0,svm,0.615270,0.476673,0.572516,0.573564,882.657005,0.349508,0.506667,0.236842,0.529412,0.500000
2,2,0,lightgbm,0.849928,0.557809,0.410791,0.562996,79.287451,0.200566,0.533333,0.218750,0.411765,0.568966
3,3,1,mlp,0.764750,0.698984,0.461688,0.629941,148.227856,0.270025,0.736842,0.480000,0.631579,0.771930
4,4,1,svm,0.668387,0.672207,0.570251,0.581209,1548.510465,0.352554,0.592105,0.363636,0.842105,0.508772
5,5,1,lightgbm,0.818336,0.721145,0.435367,0.519793,76.869776,0.245088,0.710526,0.448276,0.684211,0.719298
6,6,2,mlp,0.775135,0.582641,0.450725,0.637675,112.249024,0.230674,0.513158,0.275000,0.578947,0.491228
7,7,2,svm,0.741989,0.619575,0.544436,0.562643,1877.521160,0.233194,0.250000,0.250000,1.000000,0.000000
8,8,2,lightgbm,0.882056,0.607572,0.375743,0.609719,81.188117,0.281858,0.578947,0.259259,0.368421,0.649123
9,9,3,mlp,0.733354,0.656746,0.486908,0.569233,161.590503,0.230696,0.675676,0.406250,0.722222,0.660714
